<a href="https://colab.research.google.com/github/Marc86316/DataCamp/blob/main/certification/Practical_Exam_Spectrum_Shades_LLC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Practical Exam: Spectrum Shades LLC
Spectrum Shades LLC is a prominent supplier of concrete color solutions, offering a wide range of pigments and coloring systems used in various concrete applications, including decorative concrete, precast concrete, and concrete pavers. The company prides itself on delivering high-quality colorants that meet the unique needs of its diverse clientele, including contractors, architects, and construction companies.
</br></br>
The company has recently observed a growing number of customer complaints regarding inconsistent color quality in their products. The discrepancies have led to a decline in customer satisfaction and a potential increase in product returns.
By identifying and mitigating the factors causing color variations, the company can enhance product reliability, reduce customer complaints, and minimize return rates.
</br></br>
You are part of the data analysis team tasked with providing actionable insights to help Spectrum Shades LLC address the issues of inconsistent color quality and improve customer satisfaction.

In [14]:
import pandas as pd
import numpy as np

url = 'https://raw.githubusercontent.com/Marc86316/DataCamp/refs/heads/main/certification/production_data.csv'
# 1) 載入原始 CSV
df = pd.read_csv(url)

# Task 1

Before you can start any analysis, you need to confirm that the data is accurate and reflects what you expect to see.

It is known that there are some issues with the `production_data` table, and the data team have provided the following data description.

Write a query to ensure the data matches the description provided, including identifying and cleaning all invalid values. You must match all column names and description criteria.
</br>

- You should start with the data in the file "production_data.csv".
- Your output should be a DataFrame named clean_data.
- All column names and values should match the table below.
</br>

| Column Name             | Criteria                                                                                         |
|--------------------------|--------------------------------------------------------------------------------------------------|
| batch_id | Discrete. Identifier for each batch. Missing values are not possible. |
| production_date | Date. Date when the batch was produced.|
| raw_material_supplier | Categorical. Supplier of the raw materials. (1='national_supplier', 2='international_supplier'). <br> Missing values should be replaced with 'national_supplier'.|
| pigment_type           | Nominal. Type of pigment used. ['type_a', 'type_b', 'type_c']. <br> Missing values should be replaced with 'other'. |
| pigment_quantity       | Continuous. Amount of pigment added (in kilograms) (Range: 1 - 100). <br> Missing values should be replaced with median. |
| mixing_time           | Continuous. Duration of the mixing process (in minutes). <br> Missing values should be replaced with mean, rounded to 2 decimal places. |
| mixing_speed          | Categorical. Speed of the mixing process represented as categories: 'Low', 'Medium', 'High'.</br> Missing values should be replaced with 'Not Specified'. |
| product_quality_score | Continuous. Overall quality score of the final product (rating on a scale of 1 to 10). <br> Missing values should be replaced with mean, rounded to 2 decimal places. |


In [15]:
# 2) batch_id → int64，去除空值
df['batch_id'] = pd.to_numeric(df['batch_id'], errors='coerce')
df = df.dropna(subset=['batch_id'])
df['batch_id'] = df['batch_id'].astype('int64')

# 3) production_date → datetime，去除無效
df['production_date'] = pd.to_datetime(df['production_date'], errors='coerce')
df = df.dropna(subset=['production_date'])

# 4) raw_material_supplier → 類別 ('national_supplier','international_supplier')
supplier_map = {'1': 'national_supplier', 1: 'national_supplier',
                '2': 'international_supplier', 2: 'international_supplier'}
df['raw_material_supplier'] = df['raw_material_supplier'].replace(supplier_map)
df['raw_material_supplier'] = df['raw_material_supplier'].fillna('national_supplier')
df['raw_material_supplier'] = df['raw_material_supplier'].astype('category')

# 5) pigment_type → 類別 (type_a,b,c 或 other)
valid_types = ['type_a', 'type_b', 'type_c']
df['pigment_type'] = df['pigment_type'].astype(str).str.strip().str.lower()
df.loc[~df['pigment_type'].isin(valid_types), 'pigment_type'] = 'other'
df['pigment_type'] = df['pigment_type'].fillna('other').astype('category')

# 6) pigment_quantity → float64 (1–100)，空值補中位數
df['pigment_quantity'] = pd.to_numeric(df['pigment_quantity'], errors='coerce')
df.loc[(df['pigment_quantity'] < 1) | (df['pigment_quantity'] > 100),
       'pigment_quantity'] = np.nan
df['pigment_quantity'] = df['pigment_quantity']\
    .fillna(df['pigment_quantity'].median())\
    .astype('float64')

# 7) mixing_time → float64，空值補平均並四捨五入 2 位
df['mixing_time'] = pd.to_numeric(df['mixing_time'], errors='coerce')
df['mixing_time'] = df['mixing_time'].fillna(df['mixing_time'].mean())\
                                 .round(2).astype('float64')

# 8) mixing_speed → 類別 (Low/Medium/High/Not Specified)
valid_speeds = ['Low', 'Medium', 'High']
df['mixing_speed'] = df['mixing_speed'].astype(str).str.strip().str.title()
df.loc[~df['mixing_speed'].isin(valid_speeds), 'mixing_speed'] = 'Not Specified'
df['mixing_speed'] = df['mixing_speed'].fillna('Not Specified').astype('category')

# 9) product_quality_score → float64 (1–10)，空值補平均並四捨五入 2 位
df['product_quality_score'] = pd.to_numeric(df['product_quality_score'], errors='coerce')
df.loc[(df['product_quality_score'] < 1) | (df['product_quality_score'] > 10),
       'product_quality_score'] = np.nan
df['product_quality_score'] = df['product_quality_score']\
    .fillna(df['product_quality_score'].mean())\
    .round(2).astype('float64')

# 最終 clean_data
clean_data = df.copy()
clean_data

,batch_id,production_date,raw_material_supplier,pigment_type,pigment_quantity,mixing_time,mixing_speed,product_quality_score
0,1,2024-06-25,national_supplier,type_a,42.822881,22.56,High,7.17
1,2,2023-11-23,international_supplier,type_b,42.873479,44.97,High,6.85
2,3,2024-02-18,international_supplier,type_b,33.251206,52.62,High,5.66
3,4,2023-11-11,national_supplier,type_a,52.532817,24.96,High,6.99
4,5,2024-04-11,international_supplier,type_c,36.205108,25.37,High,7.10
...,...,...,...,...,...,...,...,...
1995,1996,2024-03-10,international_supplier,type_c,42.916500,31.59,Low,5.04
1996,1997,2024-05-10,international_supplier,type_b,32.262587,53.31,Low,7.58
1997,1998,2024-05-21,international_supplier,type_a,34.326753,16.45,Medium,6.18
1998,1999,2024-02-15,international_supplier,type_b,46.148489,54.79,Low,5.06


# Task 2

You want to understand how the supplier type and quantity of materials affect the final product attributes.

Calculate the average `product_quality_score` and `pigment_quantity` grouped by `raw_material_supplier`.

- You should start with the data in the file 'production_data.csv'.
- Your output should be a DataFrame named aggregated_data.
- It should include the three columns: `raw_material_supplier`, `avg_product_quality_score`, and `avg_pigment_quantity`.
- Your answers should be rounded to 2 decimal places.


In [19]:
import pandas as pd

# 1) 讀取原始 CSV
df = pd.read_csv(url)

# 2) 清洗 raw_material_supplier 只需對應標籤與空值補 1
supplier_map = {'1': 'national_supplier', 1: 'national_supplier',
                '2': 'international_supplier', 2: 'international_supplier'}
df['raw_material_supplier'] = df['raw_material_supplier']\
    .replace(supplier_map)\
    .fillna('national_supplier')

# 3) 轉數值
df['pigment_quantity'] = pd.to_numeric(df['pigment_quantity'], errors='coerce')
df['product_quality_score'] = pd.to_numeric(df['product_quality_score'], errors='coerce')

# 4) GroupBy & agg
aggregated_data = (
    df.groupby('raw_material_supplier', dropna=False)
      .agg(
        avg_product_quality_score = ('product_quality_score', 'mean'),
        avg_pigment_quantity      = ('pigment_quantity', 'mean')
      )
      .reset_index()
      .round(2)
)

aggregated_data


,raw_material_supplier,avg_product_quality_score,avg_pigment_quantity
0,international_supplier,5.97,34.91
1,national_supplier,8.02,44.73


# Task 3

To get more insight into the factors behind product quality, you want to filter the data to see an average product quality score for a specified set of results.

Identify the average `product_quality_score` for batches with a `raw_material_supplier` of 2 and a `pigment_quantity` greater than 35 kg.

Write a query to return the average `avg_product_quality_score` for these filtered batches. Use the original production data table, not the output of Task 2.

- You should start with the data in the file 'production_data.csv'.
- Your output should be a DataFrame named pigment_data.
- It should consist of a 1-row DataFrame with 3 columns: `raw_material_supplier`, `pigment_quantity`, and `avg_product_quality_score`.
- Your answers should be rounded to 2 decimal places where appropriate.


In [21]:
import pandas as pd

# 1) 讀取原始 CSV
df = pd.read_csv(url)

# 2) raw_material_supplier → 整數 1/2，空值補 1
df['raw_material_supplier'] = (
    pd.to_numeric(df['raw_material_supplier'], errors='coerce')
      .fillna(1)
      .astype(int)
)

# 3) 轉數值
df['pigment_quantity'] = pd.to_numeric(df['pigment_quantity'], errors='coerce')
df['product_quality_score'] = pd.to_numeric(df['product_quality_score'], errors='coerce')

# 4) 篩選：供應商 2 且 pigment_quantity > 35
filtered = df[
    (df['raw_material_supplier'] == 2) &
    (df['pigment_quantity'] > 35)
]

# 5) 計算平均品質分數
avg_score = round(filtered['product_quality_score'].mean(), 2)

# 6) 組 1-row DataFrame，pigment_quantity 固定 35.00
pigment_data = pd.DataFrame({
    'raw_material_supplier':     [2],
    'pigment_quantity':          [35.00],
    'avg_product_quality_score': [avg_score]
})

pigment_data



,raw_material_supplier,pigment_quantity,avg_product_quality_score
0,2,35.0,5.97


# Task 4

In order to proceed with further analysis later, you need to analyze how various factors relate to product quality. Start by calculating the mean and standard deviation for the following columns: `pigment_quantity`, and `product_quality_score`. </br> These statistics will help in understanding the central tendency and variability of the data related to product quality.
</br> </br >
Next, calculate the Pearson correlation coefficient between the following variables: `pigment_quantity`, and `product_quality_score`.
</br>
These correlation coefficients will provide insights into the strength and direction of the relationships between the factors and overall product quality.


- You should start with the data in the file 'production_data.csv'.
- Calculate the mean and standard deviation for the columns pigment_quantity and product_quality_score as: `product_quality_score_mean`, `product_quality_score_sd`, `pigment_quantity_mean`, `pigment_quantity_sd`.
- Calculate the Pearson correlation coefficient between pigment_quantity and product_quality_score as: `corr_coef`
- Your output should be a DataFrame named product_quality.
- It should include the columns: `product_quality_score_mean`, `product_quality_score_sd`, `pigment_quantity_mean`, `pigment_quantity_sd`, `corr_coef`.
- Ensure that your answers are rounded to 2 decimal places.


In [22]:
# 1) 取出兩欄
pq = clean_data['pigment_quantity']
pqs= clean_data['product_quality_score']

# 2) 計算均值與標準差，並四捨五入 2 位
product_quality_score_mean = round(pqs.mean(), 2)
product_quality_score_sd   = round(pqs.std(), 2)
pigment_quantity_mean      = round(pq.mean(), 2)
pigment_quantity_sd        = round(pq.std(), 2)

# 3) 皮爾森相關係數
corr_coef = round(pq.corr(pqs), 2)

# 4) 組成 1-row DataFrame
product_quality = pd.DataFrame({
    'product_quality_score_mean': [product_quality_score_mean],
    'product_quality_score_sd':   [product_quality_score_sd],
    'pigment_quantity_mean':      [pigment_quantity_mean],
    'pigment_quantity_sd':        [pigment_quantity_sd],
    'corr_coef':                  [corr_coef]
})

product_quality


,product_quality_score_mean,product_quality_score_sd,pigment_quantity_mean,pigment_quantity_sd,corr_coef
0,6.68,1.39,38.35,6.83,0.49
